In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from pprint import pprint
from pathlib import Path

path = Path('/kaggle/input/hydrangea-dataset-compressed/data_CNN')
list(path.glob('*'))

[PosixPath('/kaggle/input/hydrangea-dataset-compressed/data_CNN/test'),
 PosixPath('/kaggle/input/hydrangea-dataset-compressed/data_CNN/train')]

In [3]:
train_path = path / 'train'
test_path = path / 'test'

In [4]:
from torch.cuda import is_available

devi = 'cuda' if is_available() else 'cpu'
devi

'cpu'

### HPs
***

In [5]:
EPOCHS = 10
BATCH_SIZE = 128
LR = 0.0005

### Data
***

In [6]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, ColorJitter, GaussianBlur, RandomHorizontalFlip, RandomVerticalFlip

transf = Compose([
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    ColorJitter(0.1, 0.1, 0.1),
    GaussianBlur(1),
    Resize((95,95)),
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

img_tr = ImageFolder(train_path, transform=transf)
img_tr[0][0].shape
tr_dl = DataLoader(img_tr, 
                   batch_size=BATCH_SIZE, 
                   num_workers=2,
                   drop_last=True)

img_te = ImageFolder(test_path, transform=transf)
te_dl = DataLoader(img_tr, 
                   batch_size=BATCH_SIZE, 
                   num_workers=2,
                   drop_last=True)

torch.Size([3, 95, 95])

## Baseline
***

In [7]:
from torch import nn, numel, float as pt_float, long as pt_long, mean, as_tensor, rand
from torch.optim import Adam

In [8]:
# linear, no dropout, no batchnorm, no cnn, 

class BL_NET(nn.Module):
    def __init__(self, in_features=3*95*95):
        super().__init__()
        self.in_features = in_features
        
        self.model = nn.Sequential(
            nn.Flatten(),  # C * H * W
            nn.Linear(self.in_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
            nn.LogSoftmax(dim=-1)
        )
        
        self.count_w_b()
    
    def forward(self, x):
        return self.model(x)
    
    def count_w_b(self):
        print(f'# Params: {sum(numel(p) for p in self.parameters()):,}')
    
bl_net = BL_NET().to(devi)
#bl_net.count_w_b()
#bl_net(next(iter(tr_dl))[0]).shape

opt = Adam(bl_net.parameters(), lr=LR)
criterion = nn.NLLLoss(reduction='mean')

bl_net = bl_net.train()
for epoch in range(EPOCHS):
    running_acc = []
    running_loss = []
    for images, labels in tr_dl:
        images = images.to(devi, dtype=pt_float)
        labels = labels.to(devi, dtype=pt_long)

        out = bl_net(images)
        loss = criterion(out, labels)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        running_acc.append( (out.argmax(dim=1) == labels).sum() / len(labels) )
        running_loss.append(loss)
        
    print(f'Epoch {epoch:^3} | Accuracy {mean(as_tensor(running_acc)):^3.2f} | Loss: {mean(as_tensor(running_loss)):^3.2f}')
    
bl_net = bl_net.eval()
test_acc = []
for images, labels in te_dl:
    images = images.to(devi, dtype=pt_float)
    labels = labels.to(devi, dtype=pt_long)

    out = bl_net(images)

    test_acc.append((out.argmax(dim=1) == labels).sum() / len(labels))

print(f'Test Accuracy {mean(as_tensor(test_acc)):^3.2f}')

# Params: 1,735,010
Epoch  0  | Accuracy 0.50 | Loss: 2.22
Epoch  1  | Accuracy 0.53 | Loss: 0.67
Epoch  2  | Accuracy 0.46 | Loss: 0.72
Epoch  3  | Accuracy 0.57 | Loss: 0.70
Epoch  4  | Accuracy 0.62 | Loss: 0.66
Epoch  5  | Accuracy 0.64 | Loss: 0.65
Epoch  6  | Accuracy 0.67 | Loss: 0.63
Epoch  7  | Accuracy 0.68 | Loss: 0.61
Epoch  8  | Accuracy 0.70 | Loss: 0.59
Epoch  9  | Accuracy 0.69 | Loss: 0.60
Test Accuracy 0.75


## CNN
***

In [9]:
class INPUTS(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.inputs = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Softshrink())
        
    def forward(self, x):
        return self.inputs(x)

inputs_block = INPUTS()
inputs_block(rand(1,3,95,95)).shape

torch.Size([1, 32, 46, 46])

In [10]:
class BLOCK(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.block = nn.Sequential(
            nn.Conv2d(32, 32, 3),
            nn.Dropout2d(0.25),
            nn.BatchNorm2d(32),
            #nn.MaxPool2d(2),
            nn.Softshrink(),
            nn.ConstantPad2d(1, 0)
        )
        
    def forward(self, x):
        return self.block(x)

block = BLOCK()
block(rand(1,32,46,46)).shape


torch.Size([1, 32, 46, 46])

In [11]:
class RES_BLOCK(nn.Module):
    def __init__(self, block):
        super().__init__()
        self.block = block
        
        self.pool = nn.AvgPool2d(2)
        
    def forward(self, x):
        return self.pool(self.block(x) + x)

res_block = RES_BLOCK(block)
res_block(rand(1,32,46,46)).shape


torch.Size([1, 32, 23, 23])

In [12]:
class LINEAR(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear(800, int(800**0.5)),
            nn.Dropout(0.25),
            nn.Linear(int(800**0.5), 2),
            nn.Dropout(0.25),
            nn.LogSoftmax(dim=-1))
        
    def forward(self, x):
        return self.linear(x)

linear = LINEAR()
linear(res_block(
    res_block(
        res_block(rand(1,32,46,46))))).shape


torch.Size([1, 2])

In [13]:
# class ARM...

In [14]:
class CNN(nn.Module):
    def __init__(self, inputs_block, res_block, linear):
        super().__init__()
        
        self.inputs_block = inputs_block
        self.res_block = res_block
        self.linear = linear
        
        self.model = nn.Sequential(
            self.inputs_block,
            self.res_block,
            self.res_block,
            self.res_block,
            #self.res_block,
            #nn.Flatten()
            self.linear,
        )
        
        self.count_w_b()
    
    def forward(self, x):
        return self.model(x)
    
    def count_w_b(self):
        print(f'# Params: {sum(numel(p) for p in self.parameters()):,}')

cnn_net = CNN(inputs_block, res_block, linear).to(devi)
cnn_net(next(iter(tr_dl))[0]).shape

opt = Adam(cnn_net.parameters(), lr=LR)
criterion = nn.NLLLoss(reduction='mean')

cnn_net = cnn_net.train()
for epoch in range(25):
    running_acc = []
    running_loss = []
    for images, labels in tr_dl:
        images = images.to(devi, dtype=pt_float)
        labels = labels.to(devi, dtype=pt_long)

        out = cnn_net(images)
        loss = criterion(out, labels)
        
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        running_acc.append((out.argmax(dim=1) == labels).sum() / len(labels))
        running_loss.append(loss)
        
    print(f'Epoch {epoch:^3} | Accuracy {mean(as_tensor(running_acc)):^3.2f} | Loss (-log(softmax())): {mean(as_tensor(running_loss)):^3.2f}')
    
cnn_net = cnn_net.eval()
test_acc = []
for images, labels in te_dl:
    images = images.to(devi, dtype=pt_float)
    labels = labels.to(devi, dtype=pt_long)

    out = cnn_net(images)

    test_acc.append((out.argmax(dim=1) == labels).sum() / len(labels))

print(f'Test Accuracy {mean(as_tensor(test_acc)):^3.2f}')

# Params: 32,758


torch.Size([128, 2])

Epoch  0  | Accuracy 0.50 | Loss (-log(softmax())): 1.04
Epoch  1  | Accuracy 0.59 | Loss (-log(softmax())): 0.67
Epoch  2  | Accuracy 0.61 | Loss (-log(softmax())): 0.66
Epoch  3  | Accuracy 0.61 | Loss (-log(softmax())): 0.67
Epoch  4  | Accuracy 0.62 | Loss (-log(softmax())): 0.66
Epoch  5  | Accuracy 0.66 | Loss (-log(softmax())): 0.64
Epoch  6  | Accuracy 0.67 | Loss (-log(softmax())): 0.61
Epoch  7  | Accuracy 0.68 | Loss (-log(softmax())): 0.60
Epoch  8  | Accuracy 0.69 | Loss (-log(softmax())): 0.60
Epoch  9  | Accuracy 0.67 | Loss (-log(softmax())): 0.60
Epoch 10  | Accuracy 0.68 | Loss (-log(softmax())): 0.58
Epoch 11  | Accuracy 0.71 | Loss (-log(softmax())): 0.57
Epoch 12  | Accuracy 0.72 | Loss (-log(softmax())): 0.55
Epoch 13  | Accuracy 0.72 | Loss (-log(softmax())): 0.56
Epoch 14  | Accuracy 0.73 | Loss (-log(softmax())): 0.55
Epoch 15  | Accuracy 0.73 | Loss (-log(softmax())): 0.57
Epoch 16  | Accuracy 0.70 | Loss (-log(softmax())): 0.56
Epoch 17  | Accuracy 0.71 | Los